## Importar librerías

In [1]:
import os
import glob
import pandas as pd
import numpy as np
from genetic_algorithm import GeneticAlgorithmCV
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import KernelPCA
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import make_pipeline, Pipeline
import xgboost as xgb
from concurrent.futures import ThreadPoolExecutor, wait
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import convert_sklearn, update_registered_converter
from skl2onnx.common.shape_calculator import calculate_linear_classifier_output_shapes
from onnxmltools.convert.xgboost.operator_converters.XGBoost import convert_xgboost
import warnings
warnings.filterwarnings('ignore')

## Cargar y preparar datos

In [2]:
file_folder = r"/mnt/c/Users/Administrador/AppData/Roaming/MetaQuotes/Terminal/Common/Files/"
# Leer los archivos CSV
file_pattern = os.path.join(file_folder, 'training_dataset_*.csv')
df_file_path = glob.glob(file_pattern)
df = pd.read_csv(df_file_path[0])

In [3]:
def create_training_dataset(df, trade_type):
    df = df.drop_duplicates()
    # Filtrar las operaciones del tipo especificado y con profit != 0
    df_trade = df[(df['type'] == trade_type) & (df['profit'] != 0)].copy()
    # Añadir la columna 'Target' basada en el profit
    df_trade['target'] = df_trade['profit'].apply(lambda x: 1 if x > 0 else 0)
    # Seleccionar las columnas necesarias (todas menos las dos últimas para el conjunto principal,
    feature_columns = df.columns[:-2]
    df_trade = df_trade[feature_columns.tolist() + ['target']]
    # Eliminar posibles missings
    if(df_trade.isna().values.any()):
        df_trade = df_trade.dropna()
    n_winning = len(df_trade[df_trade['target']==1])
    n_losing = len(df_trade[df_trade['target']==0])
    print(f"Tipo de operación: {'Buy' if trade_type == 1 else 'Sell'}")
    print(f"Número de características: {df.iloc[:, :-2].shape[1]}")
    print(f"Total Ganadoras: {n_winning}")
    print(f"Total Perdedoras: {n_losing}")
    print("___________________________________")
    # return dataframe
    return df_trade

In [4]:
# X and y
def load_dataset(df):
    # Preparación de los datos de compra
    df_buy = create_training_dataset(df, trade_type=1)
    X_buy_train = df_buy.drop(columns='target')
    y_buy_train = df_buy['target']
    # Preparación de los datos de venta
    df_sell = create_training_dataset(df, trade_type=-1)
    X_sell_train = df_sell.drop(columns='target')
    y_sell_train = df_sell['target']
    return X_buy_train.values, y_buy_train.values, X_sell_train.values, y_sell_train.values

In [ ]:
# X & y data
X_buy_train, y_buy_train, X_sell_train, y_sell_train = load_dataset(df)

## Entrenamiento de los modelos

In [5]:
def create_scaler_selector(classifier, cv, X_train, y_train):
    scaler = RobustScaler()
    X_scaled = scaler.fit_transform(X_train, y_train)
    selector = RFECV(
        estimator=classifier,
        step=1,
        cv=cv,
        scoring="accuracy",
        min_features_to_select=2,
        n_jobs=-1,
    )
    X_transformed = selector.fit_transform(X_scaled, y_train)
    return scaler, selector, X_transformed

In [10]:
# Definir el clasificador
classifier = xgb.XGBClassifier(
    eval_metric='logloss',
    tree_method = "hist",
    device = "cuda",
    predictor='gpu_predictor', 
    verbosity=0
    )
# Definir algoritmo de validación cruzada estratificada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

Tipo de operación: Buy
Número de características: 76
Total Ganadoras: 2624
Total Perdedoras: 2197
___________________________________
Tipo de operación: Sell
Número de características: 76
Total Ganadoras: 2457
Total Perdedoras: 1920
___________________________________


In [11]:
# Entrenar modelos simultáneamente
with ThreadPoolExecutor(max_workers=2) as executor:
    print("Esperando que las tareas finalicen...")
    # enviar tareas de entrenamiento
    future_buy = executor.submit(create_scaler_selector, classifier, cv, X_buy_train, y_buy_train)
    future_sell = executor.submit(create_scaler_selector, classifier, cv, X_sell_train, y_sell_train)
    # esperar a que todas las tareas terminen
    futures = [future_buy, future_sell]
    wait(futures)
    # Obtener resultados una vez que ambas tareas han terminado
    scaler_buy, selector_buy, X_buy_train_transformed = future_buy.result()
    scaler_sell, selector_sell, X_sell_train_transformed = future_sell.result()
    print("¡Todas las tareas han terminado!")

Esperando que las tareas finalicen...
¡Todas las tareas han terminado!


In [12]:
# Train model functions
def train_classifier(X_train, y_train, model_type, classifier, cv, param_grid):
    try:
        # Entrenar el modelo utilizando el algoritmo genético
        ga_search = GeneticAlgorithmCV(
            model_type=model_type,
            estimator=classifier,
            cv=cv,
            param_grid=param_grid,
            n_jobs=-1,
            verbose=True
        )
        ga_search.fit(X_train, y_train)
    except Exception as e:
        print(f"Error en traing model {model_type}: {e}")
        raise
    # Obtener los mejores parámetros y el mejor estimador
    print("####################################################################")
    print(f"Mejor puntuación de validación para {model_type}: {ga_search.best_score_}")
    print(f"Mejores parámetros encontrados para {model_type}: {ga_search.best_params_}")
    print("####################################################################")
    # Retornar mejor estimador
    return ga_search.best_estimator_

In [13]:
# Definir espacio de hiperparámetros
n_features = len(df.iloc[:, :-2].columns)
param_grid = {
    #'kernelpca__n_components': {'type': 'int', 'low': 2, 'high': n_features-1},
    'xgbclassifier__n_estimators': {'type': 'int', 'low': 50, 'high': 500},
    'xgbclassifier__max_depth': {'type': 'int', 'low': 3, 'high': 50},
    'xgbclassifier__learning_rate': {'type': 'float', 'low': 0.001, 'high': 0.3},
    'xgbclassifier__subsample': {'type': 'float', 'low': 0.6, 'high': 1.0},
    'xgbclassifier__colsample_bytree': {'type': 'float', 'low': 0.6, 'high': 1.0},
    'xgbclassifier__gamma': {'type': 'float', 'low': 0.0, 'high': 0.5},
    'xgbclassifier__reg_alpha': {'type': 'float', 'low': 0.0, 'high': 1.0},
    'xgbclassifier__reg_lambda': {'type': 'float', 'low': 0.0, 'high': 1.0},
    'xgbclassifier__min_child_weight': {'type': 'int', 'low': 1, 'high': 10},
    'xgbclassifier__scale_pos_weight': {'type': 'float', 'low': 1.0, 'high': 10.0}
}

In [14]:
# Entrenar modelos simultáneamente
with ThreadPoolExecutor(max_workers=2) as executor:
    print("Esperando que las tareas finalicen...")
    # enviar tareas de entrenamiento
    future_buy = executor.submit(train_classifier, X_buy_train_transformed, y_buy_train, "buy", classifier, cv, param_grid)
    future_sell = executor.submit(train_classifier, X_sell_train_transformed, y_sell_train, "sell", classifier, cv, param_grid)
    # esperar a que todas las tareas terminen
    futures = [future_buy, future_sell]
    wait(futures)
    # Obtener resultados una vez que ambas tareas han terminado
    classifier_buy = future_buy.result()
    classifier_sell = future_sell.result()
    print("¡Todas las tareas han terminado!")

Esperando que las tareas finalicen...


Generaciones buy:   0%|          | 0/15 [00:00<?, ?gen/s]

Generaciones sell:   0%|          | 0/15 [00:00<?, ?gen/s]

[1, sell] Fitness: 0.5677110653214423 | Best Fitness: 0.5677110653214423
[1, sell] Fitness Improvement: 0.0000 | Diversity: 1.2819
[1, sell] Normalized Fitness Improvement: 0.0000 | Normalized Diversity: 0.0000
[1, sell] Crossover Rate: 0.5000 | Mutation Rate: 0.5000
[select_parents]: num_selected_elites: 2
[1, buy] Fitness: 0.5884806995509662 | Best Fitness: 0.5884806995509662
[1, buy] Fitness Improvement: 0.0000 | Diversity: 1.2770
[1, buy] Normalized Fitness Improvement: 0.0000 | Normalized Diversity: 0.0000
[1, buy] Crossover Rate: 0.5000 | Mutation Rate: 0.5000
[select_parents]: num_selected_elites: 2
[2, sell] Fitness: 0.5677110653214423 | Best Fitness: 0.5677110653214423
[2, sell] Fitness Improvement: 0.0000 | Diversity: 1.2957
[2, sell] Normalized Fitness Improvement: 0.0000 | Normalized Diversity: 1.0000
[2, sell] Crossover Rate: 0.6848 | Mutation Rate: 0.3152
[2, sell] Early stopping due to no improvement.
Best fitness for sell: 0.5677110653214423
############################

In [15]:
def create_pipeline(scaler, selector, classifier):
    pipeline = make_pipeline(
        scaler,
        selector,
        classifier
    )
    return pipeline

In [17]:
model_buy = create_pipeline(scaler=scaler_buy, selector=selector_buy, classifier=classifier_buy)
model_sell = create_pipeline(scaler=scaler_sell, selector=selector_sell, classifier=classifier_sell)

## Exportar modelos a formato ONNX

In [18]:
def save_onnx_models(mql5_files_folder):
    try:
        update_registered_converter(
            xgb.XGBClassifier,
            "XGBClassifier",
            calculate_linear_classifier_output_shapes,
            convert_xgboost,
            options={'nocl': [True, False], 'zipmap': [True, False, 'columns']}
        )
        model_buy_onnx = convert_sklearn(
            model_buy,
            'pipeline_buy_xgboost',
            [('input', FloatTensorType([None, X_buy_train.shape[1]]))],
            target_opset={'': 12, 'ai.onnx.ml': 2}
        )
        model_sell_onnx = convert_sklearn(
            model_sell,
            'pipeline_sell_xgboost',
            [('input', FloatTensorType([None, X_buy_train.shape[1]]))],
            target_opset={'': 12, 'ai.onnx.ml': 2}
        )
        with open(os.path.join(mql5_files_folder, "model_buy.onnx"), 'wb') as f:
            f.write(model_buy_onnx.SerializeToString())
        with open(os.path.join(mql5_files_folder, "model_sell.onnx"), 'wb') as f:
            f.write(model_sell_onnx.SerializeToString())
    except Exception as e:
        print(f"Error en exportar los modelos: {e}")
        raise
    print("Modelos ONNX exportados correctamente")

In [19]:
save_onnx_models(r'/mnt/c/Users/Administrador/AppData/Roaming/MetaQuotes/Terminal/6C3C6A11D1C3791DD4DBF45421BF8028/MQL5/Files')

Modelos ONNX exportados correctamente
